# Trail Recommendations

List everything that we need to do.

In [1]:
import Trailforks as tf

## Getting data from Trailforks

In [2]:
search = tf.trailforksSearch()
data = search.fetchData()
data.head()
data['trailtype']

0    1
1    1
2    2
Name: trailtype, dtype: object

In [4]:
search.fetchDataByTrailType("1")

,trailid,vid,title,rid,difficulty,activitytype,trailtype,biketype,physical_rating,ttfs,...,act_skialpine,act_skixc,act_skibc,act_horse,act_snowmobile,act_mototrials,latitude,longitude,staticmap_image1,activitytypes
0,3,2,Floppy Bunny,1,5,1,1,"1,2",2,"16,15",...,0,0,0,0,0,0,49.359050,-123.040270,https://ep1.pinkbike.org/trailstaticmap/0/3_2_...,"[1, 2, 6, 5]"
1,5,8,Pipeline,1,5,1,1,"1,2",2,"54,5,15,19",...,0,0,0,0,0,0,49.358990,-123.049780,https://ep1.pinkbike.org/trailstaticmap/0/5_8_...,"[1, 6, 5]"
2,6,5,Bobsled,1,4,1,2,"1,2,7",1,"1,54,17,104,21",...,0,0,0,0,0,0,49.358250,-123.040890,https://ep1.pinkbike.org/trailstaticmap/0/6_5_...,[1]
